## P2 Exploration

Basic exploration will be done in this jupyter notebook.

In [ ]:
from helpers import *
import pandas as pd
 
# Just fill this with wherever is your main data folder is (copy the path just before 'MovieSummaries'):
file_source = "/Users/shrinidhivelan/Downloads/"

# Same goes for the reviews dataset : but simply copy the entire path of where your reviews data is... 
reviews_path = "/Users/shrinidhivelan/Downloads/TMDB_movie_dataset_v11.csv"

# replace the following by the direct path of where your bechdel test movie data is:
bechdel_path = "/Users/shrinidhivelan/Downloads/movies.csv"

### creating dataset by extracting 
MovieMetadata_df, CharacterMetadata_df, names_df, plot_summaries_df, tvTropes_df, merged_df = create_datasets(file_source, reviews_path)




In [20]:
# The bechdel data contains ~1700 lines, to be separated into train and test sets in order to create a model to predict the binary result
#### 80% train and 20% test for instance, this will be used to merge main data into this : 
bechdel_data = pd.read_csv(bechdel_path)
bechdel_data = bechdel_data[['year', 'title', 'binary','budget']]


In [21]:
bechdel_data_merged = bechdel_data.merge(MovieMetadata_df, how='left', left_on = 'title', right_on = 'Movie name')


In [22]:
bechdel_data

,year,title,binary,budget
0,2013,21 &amp; Over,FAIL,13000000
1,2012,Dredd 3D,PASS,45000000
2,2013,12 Years a Slave,FAIL,20000000
3,2013,2 Guns,FAIL,61000000
4,2013,42,FAIL,40000000
...,...,...,...,...
1789,1971,Shaft,FAIL,53012938
1790,1971,Straw Dogs,FAIL,25000000
1791,1971,The French Connection,FAIL,2200000
1792,1971,Willy Wonka &amp; the Chocolate Factory,FAIL,3000000


In [23]:
bechdel_data_merged

,year,title,binary,budget,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples),Year
0,2013,21 &amp; Over,FAIL,13000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,Dredd 3D,PASS,45000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,12 Years a Slave,FAIL,20000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,2 Guns,FAIL,61000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,42,FAIL,40000000,36239136.0,/m/0k2jpkj,42,2013-04-12,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01z02hx"": ""Sports"", ""/m/03bxz7"": ""Biograp...",2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,1971,Straw Dogs,FAIL,25000000,196176.0,/m/01bwgr,Straw Dogs,1971-11-03,11148828.0,118.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama""}",1971.0
2343,1971,Straw Dogs,FAIL,25000000,29657080.0,/m/084nr2g,Straw Dogs,2011-09-16,10324441.0,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/02wtdps"": ""Crime...",2011.0
2344,1971,The French Connection,FAIL,2200000,99463.0,/m/0pd64,The French Connection,1971-10-07,51700000.0,104.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/0lsxr"": ""Crime F...",1971.0
2345,1971,Willy Wonka &amp; the Chocolate Factory,FAIL,3000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
